In [ ]:
from pathlib import Path
import pandas as pd
import random
import os
import datetime

In [ ]:
def rowstack_dataframes(paths, outfile_path):
    dfs = []
    for path in paths:
        dfs.append(pd.read_csv(path))
    df = pd.concat(dfs)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(outfile_path, index=False, sep="\t")

In [ ]:
# Create the full dataset of genes and text snippets.
name = "../genes_snps_snippets.tsv"
paths = [
    "../data/S236523_snps_and_snippets_09_25_2020_h14m27s12.csv",
    "../data/S236523_snps_and_snippets_09_25_2020_h14m27s12.csv",
]
rowstack_dataframes(paths,name)





In [ ]:
# Create the full dataset of genes and context sentences.
name = "../genes_snps_contexts.tsv"
paths = [
    "../data/S236523_snps_and_contexts_09_25_2020_h14m27s12.csv",
    "../data/S236523_snps_and_contexts_09_25_2020_h14m27s12.csv",
]
rowstack_dataframes(paths,name)


